In [1]:

bucket= "output-bucket-default-1wuspi8x"

import pyspark

In [2]:

patients = spark.read.parquet("s3a://%s/patients/_raw" %(bucket))

print(patients.count())
patients.printSchema()


22/11/21 20:17:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/11/21 20:17:27 WARN DataSource: All paths were ignored:
  s3a://output-bucket-default-1wuspi8x/patients/_raw


1171
root
 |-- id: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- death_date: string (nullable = true)
 |-- social_sec_num: string (nullable = true)
 |-- drivers: string (nullable = true)
 |-- passport: string (nullable = true)
 |-- prefix: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- suffix: string (nullable = true)
 |-- maiden_name: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- race: string (nullable = true)
 |-- ethnicity: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birthplace: string (nullable = true)
 |-- address: string (nullable = true)
 |-- postal_code: long (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- county: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- healthcare_coverage: double (nullable = true)
 |-- healthcare_expenses: double (nulla

In [3]:

from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import Window as w
from pyspark.sql.functions import udf

def ascii_ignore(x):
    return x.encode('ascii', 'ignore').decode('ascii')

ascii_udf = udf(ascii_ignore)

def trim_to_null(c):
  return (
    f.lower(
      f.when(f.trim(f.col(c)) == '', None)
      .when(f.trim(f.col(c)) == 'null', None)
      .otherwise(f.trim(f.col(c)))
    )
  )

keep_cols = ['id', 'birth_date', 'first_name', 'surname', 'social_sec_num','suffix']

people_bronze = patients.select(*keep_cols)
people_bronze = people_bronze.withColumn("first_name", ascii_udf('first_name'))
people_bronze = people_bronze.withColumn("last_name", ascii_udf('surname'))
people_bronze = people_bronze.withColumn("suffix", f.lower(trim_to_null("suffix")))
people_bronze = people_bronze.drop(people_bronze.surname)
people_bronze.printSchema()
people_bronze.write.format("parquet").mode("overwrite").save("s3a://%s/patients/_bronze" %(bucket))

root
 |-- id: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- social_sec_num: string (nullable = true)
 |-- suffix: string (nullable = true)
 |-- last_name: string (nullable = true)



In [ ]:
people_bronze.write.mode("overwrite").saveAsTable("patients")

In [4]:
sc = spark.sparkContext
sql_context = SQLContext(sc)

distinct_id = sql_context.sql("SELECT distinct id FROM patients").count()
print(distinct_id)

distinct_all = sql_context.sql("SELECT id, first_name, last_name, birth_date, social_sec_num FROM patients").count()
print(distinct_all)

distinct_ssn = sql_context.sql("SELECT distinct social_sec_num FROM patients").count()
print(distinct_ssn)

/home/linuxbrew/.linuxbrew/Cellar/apache-spark/3.3.1/libexec/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


22/11/21 20:18:51 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/11/21 20:18:51 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/11/21 20:18:55 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/11/21 20:18:55 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore jon@127.0.1.1
1171
1171
1070


In [5]:
%pip install pandas

people_bronze.toPandas()

Note: you may need to restart the kernel to use updated packages.


,id,birth_date,first_name,social_sec_num,suffix,last_name
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,5/25/1989,Jos Eduardo181,999-76-6866,None,Gmez206
1,034e9e3b-2def-4559-bb2a-7850888ae060,11/14/1983,Milo271,999-99-8899,None,Feil794
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,6/2/1992,Jayson808,999-27-3385,None,Fadel536
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,5/27/1978,Mariana775,999-61-7267,None,Rutherford999
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,10/18/1996,Gregorio366,999-60-7372,None,Auer97
...,...,...,...,...,...,...
1166,023a7d29-32b3-4db5-89c8-b88bd7582ec0,2/14/1922,Conchita9,999-60-9291,None,Peres371
1167,1fc8dafd-90e5-44d6-9179-81156c0ea083,8/8/1977,Donn979,999-29-3501,None,Casper496
1168,6d048a56-edb8-4f29-891d-7a84d75a8e78,9/5/1914,Kimber624,999-60-2184,None,Volkman526
1169,fca3178e-fb68-41c3-8598-702d3ca68b96,9/5/1914,Shira43,999-48-3257,None,Bruen238
